In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm
from pprint import pprint

In [7]:
import os
import tensorflow_hub as hub
import tensorflow as tf

LABEL_INT_DICT = np.unique(pd.read_csv('ucf11_train_vid.csv')['label'])
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}
pprint(LABEL_INT_DICT)
pprint(LABEL_STR_DICT)

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",
                   trainable=True),  # Can be True, see below.
    tf.keras.layers.Dense(len(LABEL_INT_DICT), activation='softmax')
])
model.build([None, 256, 256, 3])  # Batch input shape.

adam = keras.optimizers.Adam(lr=0.0001)
model.compile(
    optimizer=adam,
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.summary()

array(['biking', 'diving', 'golf', 'juggle', 'jumping', 'riding',
       'shooting', 'spiking', 'swing', 'tennis', 'walk'], dtype=object)
{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              5919312   
                                                                 
 dense (Dense)               (None, 11)                14091     
                                                                 
Total params: 5,933,403
Trainable params: 5,872,795
Non-trainable params: 60,608
_________________________________________________________________


C:\Users\khj_a\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [2]:
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow import keras

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
model = keras.models.load_model(
    'ucf11_weights/10-0.98.hdf5',
    custom_objects={'KerasLayer': hub.KerasLayer}
)

model.summary()

OSError: No file or directory found at ucf11_weights/10-0.98.hdf5

# RNN  
* 입력과 출력을 시퀀스 단위로 처리하는 시퀀스 모델  
* 순환 신경망(Recurrent Neural Network)  

In [8]:
feature_extractor = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        model.layers[0]
    ], name='feature_extractor'
)

feature_extractor.summary()

Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              5919312   
                                                                 
Total params: 5,919,312
Trainable params: 5,858,704
Non-trainable params: 60,608
_________________________________________________________________


In [10]:
os.mkdir('UCF11_updated_npy')
os.mkdir('UCF11_updated_npy/train')
os.mkdir('UCF11_updated_npy/valid')

In [11]:
max_frames = 10
SAVE_DIR = 'UCF11_updated_npy/'
train_df = pd.read_csv('ucf11_train_vid.csv')
valid_df = pd.read_csv('ucf11_valid_vid.csv')

In [12]:
for i, elem in tqdm(train_df.iterrows(),
                   total=len(train_df)):
    label = elem['label']
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.

        frames.append(frame_)
        if len(frames) == max_frames:
            break
    cap.release()
    
    frames = np.array(frames)
    features = feature_extractor.predict(frames)
    
    file_name = SAVE_DIR + f'train/{label}_{i}.npy'
    np.save(file_name, features)

100%|██████████| 220/220 [02:01<00:00,  1.81it/s]


In [13]:
print(len(glob(SAVE_DIR + 'train/*')))

220


In [14]:
for i, elem in tqdm(valid_df.iterrows(),
                   total=len(valid_df)):
    label = elem['label']
    cap = cv2.VideoCapture(
        elem['file_path']
    )
    
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_ = frame.copy()
        frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        frame_ = cv2.resize(frame_, (256, 256))
        frame_ = frame_.astype('float32')
        frame_ = frame_ / 255.

        frames.append(frame_)
        if len(frames) == max_frames:
            break
    cap.release()
    
    frames = np.array(frames)
    features = feature_extractor.predict(frames)
    
    file_name = SAVE_DIR + f'valid/{label}_{i}.npy'
    np.save(file_name, features)

100%|██████████| 55/55 [00:30<00:00,  1.82it/s]


In [15]:
print(len(glob(SAVE_DIR + 'valid/*')))

55
